In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_sd.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,792074,3507637205,4/27/20,876,PPP,"EMPIRICAL FOODS, INC.",891 TWO RIVERS DR,DAKOTA DUNES,SD,57049.0,...,311 Food,3116 Animal Slaughtering and Processing,3116.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,NaN
1,792075,1221237209,4/15/20,876,PPP,COCA-COLA BOTTLING COMPANY HIGH COUNTRY,2150 COCA COLA LN,RAPID CITY,SD,57702.0,...,312 Beverage and Tobacco Products,3121 Beverage Manufacturing,3121.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,NaN
2,792076,4673147200,4/27/20,876,PPP,"DAKOTA PROVISIONS, LLC",40253 US Highway 14,Huron,SD,57350.0,...,311 Food,3116 Animal Slaughtering and Processing,3116.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,NaN
3,792077,6663527703,5/1/20,876,PPP,SONIFI SOLUTIONS INC,3900 W Innovation St,Sioux Falls,SD,57107.0,...,999 Other,5121 Motion Picture and Video Industries,5121.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,NaN
4,792078,7971747009,4/8/20,876,PPP,SHOWPLACE WOOD PRODUCTS,1 Enterprise St,HARRISBURG,SD,57032.0,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,794764,8914837010,4/9/20,876,PPP,EKHOLM ENTERPRISES INCORPORATED,1101 West Glen Eagle Circle,Sioux Falls,SD,57108.0,...,999 Other,5621 Waste Collection,5621.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1101 West Glen Eagle Circle, Sioux Falls, SD",43.478080,-96.736384,NaN
2691,794765,9178718708,4/8/21,876,PPS,CARLSON'S NORTHSIDE SERVICE INC.,320 Dakota Ave N,Huron,SD,57350.0,...,999 Other,4471 Gasoline Stations,4471.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"320 Dakota Ave N, Huron, SD",44.369881,-98.213651,NaN
2692,794766,9304517103,4/15/20,876,PPP,"HART RANCHING CAMPING RESORT CLUB, INC",23756 ARENA DR,RAPID CITY,SD,57702.0,...,999 Other,7212 RV (Recreational Vehicle) Parks and Recre...,7212.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"23756 ARENA DR, RAPID CITY, SD",43.961714,-103.213474,NaN
2693,794767,9378667203,4/28/20,876,PPP,"TRADEMARK UNIFORMS, INC.",2011 W 45TH ST,SIOUX FALLS,SD,57105.0,...,999 Other,8123 Drycleaning and Laundry Services,8123.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"2011 W 45TH ST, SIOUX FALLS, SD",43.511243,-96.750715,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,3507637205,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281
1,1221237209,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592
2,4673147200,"40253 US Highway 14, Huron, SD",44.369109,-98.163177
3,6663527703,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704
4,7971747009,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/SD_fips_scraped.csv") 
fips_check_scraped.head() #15869

In [27]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [28]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

<h4>FIPS Script

In [6]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [7]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
#driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
#executable_path = {'executable_path': webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))}
executable_path = {'executable_path': ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()}
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', **executable_path, options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('46')] #SD
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

[WDM] - Downloading: 100%|██████████| 8.79M/8.79M [00:01<00:00, 8.16MB/s]


  0%|          | 0/2695 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [8]:
geo_id_list =geo_id 

Length Check

In [9]:
print(len(geo_id_list))

2695


Matching FIPS to DataFrame

In [10]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:25062]

In [11]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,3507637205,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,[461270203011033]
1,1221237209,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,[461030114003026]
2,4673147200,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,[460059568001132]
3,6663527703,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,[460990104051004]
4,7971747009,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,[460830101073026]
...,...,...,...,...,...
2690,8914837010,"1101 West Glen Eagle Circle, Sioux Falls, SD",43.478080,-96.736384,[460830101162011]
2691,9178718708,"320 Dakota Ave N, Huron, SD",44.369881,-98.213651,[460059568002064]
2692,9304517103,"23756 ARENA DR, RAPID CITY, SD",43.961714,-103.213474,[461030110052001]
2693,9378667203,"2011 W 45TH ST, SIOUX FALLS, SD",43.511243,-96.750715,[460990015013029]


Remove brackets

In [12]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,3507637205,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,461270203011033
1,1221237209,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,461030114003026
2,4673147200,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,460059568001132
3,6663527703,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,460990104051004
4,7971747009,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,460830101073026
...,...,...,...,...,...
2690,8914837010,"1101 West Glen Eagle Circle, Sioux Falls, SD",43.478080,-96.736384,460830101162011
2691,9178718708,"320 Dakota Ave N, Huron, SD",44.369881,-98.213651,460059568002064
2692,9304517103,"23756 ARENA DR, RAPID CITY, SD",43.961714,-103.213474,461030110052001
2693,9378667203,"2011 W 45TH ST, SIOUX FALLS, SD",43.511243,-96.750715,460990015013029


In [13]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
389,5274317004,"12525 US Hwy 20, BUFFALO, SD",42.365652,-97.560766,NaN
1301,9903297002,"12171 Lake Rd, BROWNS VALLEY, SD",39.242114,-121.409130,NaN


Parse for Merge

In [14]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,3507637205,461270203011033
1,1221237209,461030114003026
2,4673147200,460059568001132
3,6663527703,460990104051004
4,7971747009,460830101073026
...,...,...
2690,8914837010,460830101162011
2691,9178718708,460059568002064
2692,9304517103,461030110052001
2693,9378667203,460990015013029


Read in base data

In [15]:
#if from geocode file
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_sd.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head()

#if from FIPS file
#fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/SD_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
#fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,792074,792074,792074,3507637205,4/27/20,876,PPP,"EMPIRICAL FOODS, INC.",891 TWO RIVERS DR,DAKOTA DUNES,...,311 Food,3116 Animal Slaughtering and Processing,3116.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,NaN
1,792075,792075,792075,1221237209,4/15/20,876,PPP,COCA-COLA BOTTLING COMPANY HIGH COUNTRY,2150 COCA COLA LN,RAPID CITY,...,312 Beverage and Tobacco Products,3121 Beverage Manufacturing,3121.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,NaN
2,792076,792076,792076,4673147200,4/27/20,876,PPP,"DAKOTA PROVISIONS, LLC",40253 US Highway 14,Huron,...,311 Food,3116 Animal Slaughtering and Processing,3116.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,NaN
3,792077,792077,792077,6663527703,5/1/20,876,PPP,SONIFI SOLUTIONS INC,3900 W Innovation St,Sioux Falls,...,999 Other,5121 Motion Picture and Video Industries,5121.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,NaN
4,792078,792078,792078,7971747009,4/8/20,876,PPP,SHOWPLACE WOOD PRODUCTS,1 Enterprise St,HARRISBURG,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,NaN


Merge with MainFrame

In [16]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,792074,792074,792074,3507637205,4/27/20,876,PPP,"EMPIRICAL FOODS, INC.",891 TWO RIVERS DR,DAKOTA DUNES,...,3116 Animal Slaughtering and Processing,3116.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,NaN,461270203011033
1,792075,792075,792075,1221237209,4/15/20,876,PPP,COCA-COLA BOTTLING COMPANY HIGH COUNTRY,2150 COCA COLA LN,RAPID CITY,...,3121 Beverage Manufacturing,3121.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,NaN,461030114003026
2,792076,792076,792076,4673147200,4/27/20,876,PPP,"DAKOTA PROVISIONS, LLC",40253 US Highway 14,Huron,...,3116 Animal Slaughtering and Processing,3116.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,NaN,460059568001132
3,792077,792077,792077,6663527703,5/1/20,876,PPP,SONIFI SOLUTIONS INC,3900 W Innovation St,Sioux Falls,...,5121 Motion Picture and Video Industries,5121.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,NaN,460990104051004
4,792078,792078,792078,7971747009,4/8/20,876,PPP,SHOWPLACE WOOD PRODUCTS,1 Enterprise St,HARRISBURG,...,3371 Household and Institutional Furniture and...,3371.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,NaN,460830101073026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,794764,794764,794764,8914837010,4/9/20,876,PPP,EKHOLM ENTERPRISES INCORPORATED,1101 West Glen Eagle Circle,Sioux Falls,...,5621 Waste Collection,5621.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1101 West Glen Eagle Circle, Sioux Falls, SD",43.478080,-96.736384,NaN,460830101162011
2691,794765,794765,794765,9178718708,4/8/21,876,PPS,CARLSON'S NORTHSIDE SERVICE INC.,320 Dakota Ave N,Huron,...,4471 Gasoline Stations,4471.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"320 Dakota Ave N, Huron, SD",44.369881,-98.213651,NaN,460059568002064
2692,794766,794766,794766,9304517103,4/15/20,876,PPP,"HART RANCHING CAMPING RESORT CLUB, INC",23756 ARENA DR,RAPID CITY,...,7212 RV (Recreational Vehicle) Parks and Recre...,7212.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"23756 ARENA DR, RAPID CITY, SD",43.961714,-103.213474,NaN,461030110052001
2693,794767,794767,794767,9378667203,4/28/20,876,PPP,"TRADEMARK UNIFORMS, INC.",2011 W 45TH ST,SIOUX FALLS,...,8123 Drycleaning and Laundry Services,8123.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"2011 W 45TH ST, SIOUX FALLS, SD",43.511243,-96.750715,NaN,460990015013029


<b>Fill FIPS Columns - if FIPS file

In [17]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,792074,792074,792074,3507637205,4/27/20,876,PPP,"EMPIRICAL FOODS, INC.",891 TWO RIVERS DR,DAKOTA DUNES,...,3116.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,NaN,461270203011033,461270203011033
1,792075,792075,792075,1221237209,4/15/20,876,PPP,COCA-COLA BOTTLING COMPANY HIGH COUNTRY,2150 COCA COLA LN,RAPID CITY,...,3121.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,NaN,461030114003026,461030114003026
2,792076,792076,792076,4673147200,4/27/20,876,PPP,"DAKOTA PROVISIONS, LLC",40253 US Highway 14,Huron,...,3116.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,NaN,460059568001132,460059568001132
3,792077,792077,792077,6663527703,5/1/20,876,PPP,SONIFI SOLUTIONS INC,3900 W Innovation St,Sioux Falls,...,5121.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,NaN,460990104051004,460990104051004
4,792078,792078,792078,7971747009,4/8/20,876,PPP,SHOWPLACE WOOD PRODUCTS,1 Enterprise St,HARRISBURG,...,3371.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,NaN,460830101073026,460830101073026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,794764,794764,794764,8914837010,4/9/20,876,PPP,EKHOLM ENTERPRISES INCORPORATED,1101 West Glen Eagle Circle,Sioux Falls,...,5621.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1101 West Glen Eagle Circle, Sioux Falls, SD",43.478080,-96.736384,NaN,460830101162011,460830101162011
2691,794765,794765,794765,9178718708,4/8/21,876,PPS,CARLSON'S NORTHSIDE SERVICE INC.,320 Dakota Ave N,Huron,...,4471.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"320 Dakota Ave N, Huron, SD",44.369881,-98.213651,NaN,460059568002064,460059568002064
2692,794766,794766,794766,9304517103,4/15/20,876,PPP,"HART RANCHING CAMPING RESORT CLUB, INC",23756 ARENA DR,RAPID CITY,...,7212.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"23756 ARENA DR, RAPID CITY, SD",43.961714,-103.213474,NaN,461030110052001,461030110052001
2693,794767,794767,794767,9378667203,4/28/20,876,PPP,"TRADEMARK UNIFORMS, INC.",2011 W 45TH ST,SIOUX FALLS,...,8123.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"2011 W 45TH ST, SIOUX FALLS, SD",43.511243,-96.750715,NaN,460990015013029,460990015013029


Drop Extra FIPS Columns

In [18]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,792074,792074,792074,3507637205,4/27/20,876,PPP,"EMPIRICAL FOODS, INC.",891 TWO RIVERS DR,DAKOTA DUNES,...,311 Food,3116 Animal Slaughtering and Processing,3116.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,461270203011033
1,792075,792075,792075,1221237209,4/15/20,876,PPP,COCA-COLA BOTTLING COMPANY HIGH COUNTRY,2150 COCA COLA LN,RAPID CITY,...,312 Beverage and Tobacco Products,3121 Beverage Manufacturing,3121.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,461030114003026
2,792076,792076,792076,4673147200,4/27/20,876,PPP,"DAKOTA PROVISIONS, LLC",40253 US Highway 14,Huron,...,311 Food,3116 Animal Slaughtering and Processing,3116.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,460059568001132
3,792077,792077,792077,6663527703,5/1/20,876,PPP,SONIFI SOLUTIONS INC,3900 W Innovation St,Sioux Falls,...,999 Other,5121 Motion Picture and Video Industries,5121.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,460990104051004
4,792078,792078,792078,7971747009,4/8/20,876,PPP,SHOWPLACE WOOD PRODUCTS,1 Enterprise St,HARRISBURG,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,460830101073026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,794764,794764,794764,8914837010,4/9/20,876,PPP,EKHOLM ENTERPRISES INCORPORATED,1101 West Glen Eagle Circle,Sioux Falls,...,999 Other,5621 Waste Collection,5621.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1101 West Glen Eagle Circle, Sioux Falls, SD",43.478080,-96.736384,460830101162011
2691,794765,794765,794765,9178718708,4/8/21,876,PPS,CARLSON'S NORTHSIDE SERVICE INC.,320 Dakota Ave N,Huron,...,999 Other,4471 Gasoline Stations,4471.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"320 Dakota Ave N, Huron, SD",44.369881,-98.213651,460059568002064
2692,794766,794766,794766,9304517103,4/15/20,876,PPP,"HART RANCHING CAMPING RESORT CLUB, INC",23756 ARENA DR,RAPID CITY,...,999 Other,7212 RV (Recreational Vehicle) Parks and Recre...,7212.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"23756 ARENA DR, RAPID CITY, SD",43.961714,-103.213474,461030110052001
2693,794767,794767,794767,9378667203,4/28/20,876,PPP,"TRADEMARK UNIFORMS, INC.",2011 W 45TH ST,SIOUX FALLS,...,999 Other,8123 Drycleaning and Laundry Services,8123.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"2011 W 45TH ST, SIOUX FALLS, SD",43.511243,-96.750715,460990015013029


In [19]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
389,792463,792463,792463,5274317004,4/5/20,876,PPP,"TIM W OLSON CONSTRUCTION, INC",12525 US Hwy 20,BUFFALO,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,SOUTH DAKOTA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"12525 US Hwy 20, BUFFALO, SD",42.365652,-97.560766,NaN
1301,793375,793375,793375,9903297002,4/9/20,876,PPP,EASTMAN INC,12171 Lake Rd,BROWNS VALLEY,...,999 Other,4841 General Freight Trucking,4841.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"12171 Lake Rd, BROWNS VALLEY, SD",39.242114,-121.409130,NaN


Convert to File

In [20]:
fips_SD = fips_merge_drop

In [21]:
fips_SD.to_csv("../../data/state_data/geo/geo_fips/SD_fips_scraped.csv") 

Review & Compare

In [22]:
geo_sd_fips = pd.read_csv("../../data/state_data/geo/geo_fips/SD_fips_scraped.csv") 
geo_sd_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,792074,792074,792074,3507637205,4/27/20,876,PPP,"EMPIRICAL FOODS, INC.",891 TWO RIVERS DR,...,311 Food,3116 Animal Slaughtering and Processing,3116.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"891 TWO RIVERS DR, DAKOTA DUNES, SD",42.507600,-96.478281,4.612702e+14
1,1,792075,792075,792075,1221237209,4/15/20,876,PPP,COCA-COLA BOTTLING COMPANY HIGH COUNTRY,2150 COCA COLA LN,...,312 Beverage and Tobacco Products,3121 Beverage Manufacturing,3121.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"2150 COCA COLA LN, RAPID CITY, SD",44.106394,-103.258592,4.610301e+14
2,2,792076,792076,792076,4673147200,4/27/20,876,PPP,"DAKOTA PROVISIONS, LLC",40253 US Highway 14,...,311 Food,3116 Animal Slaughtering and Processing,3116.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"40253 US Highway 14, Huron, SD",44.369109,-98.163177,4.600596e+14
3,3,792077,792077,792077,6663527703,5/1/20,876,PPP,SONIFI SOLUTIONS INC,3900 W Innovation St,...,999 Other,5121 Motion Picture and Video Industries,5121.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"3900 W Innovation St, Sioux Falls, SD",43.578588,-96.778704,4.609901e+14
4,4,792078,792078,792078,7971747009,4/8/20,876,PPP,SHOWPLACE WOOD PRODUCTS,1 Enterprise St,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,SOUTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"1 Enterprise St, HARRISBURG, SD",43.439920,-96.703415,4.608301e+14


In [23]:
##### fill in missing Lat, Long, FIPS
geo_sd_fips.loc[geo_sd_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
389,389,792463,792463,792463,5274317004,4/5/20,876,PPP,"TIM W OLSON CONSTRUCTION, INC",12525 US Hwy 20,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,SOUTH DAKOTA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"12525 US Hwy 20, BUFFALO, SD",42.365652,-97.560766,NaN
1301,1301,793375,793375,793375,9903297002,4/9/20,876,PPP,EASTMAN INC,12171 Lake Rd,...,999 Other,4841 General Freight Trucking,4841.0,SOUTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"12171 Lake Rd, BROWNS VALLEY, SD",39.242114,-121.409130,NaN
